In [2]:
import pandas as pd
class PSAR:

  def __init__(self, init_af=0.02, max_af=0.2, af_step=0.02):
    self.max_af = max_af
    self.init_af = init_af
    self.af = init_af
    self.af_step = af_step
    self.extreme_point = None
    self.high_price_trend = []
    self.low_price_trend = []
    self.high_price_window = deque(maxlen=2)
    self.low_price_window = deque(maxlen=2)

    # Lists to track results
    self.psar_list = []
    self.af_list = []
    self.ep_list = []
    self.high_list = []
    self.low_list = []
    self.trend_list = []
    self._num_days = 0

  def calcPSAR(self, high, low):
    if self._num_days >= 3:
      psar = self._calcPSAR()
    else:
      psar = self._initPSARVals(high, low)

    psar = self._updateCurrentVals(psar, high, low)
    self._num_days += 1

    return psar

  def _initPSARVals(self, high, low):
    if len(self.low_price_window) <= 1:
      self.trend = None
      self.extreme_point = high
      return None

    if self.high_price_window[0] < self.high_price_window[1]:
      self.trend = 1
      psar = min(self.low_price_window)
      self.extreme_point = max(self.high_price_window)
    else: 
      self.trend = 0
      psar = max(self.high_price_window)
      self.extreme_point = min(self.low_price_window)

    return psar

  def _calcPSAR(self):
    prev_psar = self.psar_list[-1]
    if self.trend == 1: # Up
      psar = prev_psar + self.af * (self.extreme_point - prev_psar)
      psar = min(psar, min(self.low_price_window))
    else:
      psar = prev_psar - self.af * (prev_psar - self.extreme_point)
      psar = max(psar, max(self.high_price_window))

    return psar

  def _updateCurrentVals(self, psar, high, low):
    if self.trend == 1:
      self.high_price_trend.append(high)
    elif self.trend == 0:
      self.low_price_trend.append(low)

    psar = self._trendReversal(psar, high, low)

    self.psar_list.append(psar)
    self.af_list.append(self.af)
    self.ep_list.append(self.extreme_point)
    self.high_list.append(high)
    self.low_list.append(low)
    self.high_price_window.append(high)
    self.low_price_window.append(low)
    self.trend_list.append(self.trend)

    return psar

  def _trendReversal(self, psar, high, low):
    # Checks for reversals
    reversal = False
    if self.trend == 1 and psar > low:
      self.trend = 0
      psar = max(self.high_price_trend)
      self.extreme_point = low
      reversal = True
    elif self.trend == 0 and psar < high:
      self.trend = 1
      psar = min(self.low_price_trend)
      self.extreme_point = high
      reversal = True

    if reversal:
      self.af = self.init_af
      self.high_price_trend.clear()
      self.low_price_trend.clear()
    else:
        if high > self.extreme_point and self.trend == 1:
          self.af = min(self.af + self.af_step, self.max_af)
          self.extreme_point = high
        elif low < self.extreme_point and self.trend == 0:
          self.af = min(self.af + self.af_step, self.max_af)
          self.extreme_point = low

    return psar

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import talib as ta
from scipy.signal import argrelextrema
from collections import deque

def rsiuse(dfa1):
    global data,hh,lh,ll,hl,hh_idx,lh_idx,ll_idx,hl_idx,price,dates
    data=dfa1
    price = data['Close'].values
    dates = data.index
    # Get higher highs, lower lows, etc.
    order = 5
    hh = getHigherHighs(price, order)
    lh = getLowerHighs(price, order)
    ll = getLowerLows(price, order)
    hl = getHigherLows(price, order)
    # Get confirmation indices
    hh_idx = np.array([i[1] + order for i in hh])
    lh_idx = np.array([i[1] + order for i in lh])
    ll_idx = np.array([i[1] + order for i in ll])
    hl_idx = np.array([i[1] + order for i in hl])
    return data
def getHigherLows(data: np.array, order=5, K=2):
  # Get lows
  low_idx = argrelextrema(data, np.less, order=order)[0]
  lows = data[low_idx]
  # Ensure consecutive lows are higher than previous lows
  extrema = []
  ex_deque = deque(maxlen=K)
  for i, idx in enumerate(low_idx):
    if i == 0:
      ex_deque.append(idx)
      continue
    if lows[i] < lows[i-1]:
      ex_deque.clear()

    ex_deque.append(idx)
    if len(ex_deque) == K:
      extrema.append(ex_deque.copy())

  return extrema

def getLowerHighs(data: np.array, order=5, K=2):
  # Get highs
  high_idx = argrelextrema(data, np.greater, order=order)[0]
  highs = data[high_idx]
  # Ensure consecutive highs are lower than previous highs
  extrema = []
  ex_deque = deque(maxlen=K)
  for i, idx in enumerate(high_idx):
    if i == 0:
      ex_deque.append(idx)
      continue
    if highs[i] > highs[i-1]:
      ex_deque.clear()

    ex_deque.append(idx)
    if len(ex_deque) == K:
      extrema.append(ex_deque.copy())

  return extrema

def getHigherHighs(data: np.array, order=5, K=2):
  # Get highs
  high_idx = argrelextrema(data, np.greater, order=order)[0]
  highs = data[high_idx]
  # Ensure consecutive highs are higher than previous highs
  extrema = []
  ex_deque = deque(maxlen=K)
  for i, idx in enumerate(high_idx):
    if i == 0:
      ex_deque.append(idx)
      continue
    if highs[i] < highs[i-1]:
      ex_deque.clear()
    ex_deque.append(idx)
    if len(ex_deque) == K:
      extrema.append(ex_deque.copy())
  
  return extrema

def getLowerLows(data: np.array, order=5, K=2):
  # Get lows
  low_idx = argrelextrema(data, np.less, order=order)[0]
  lows = data[low_idx]
  # Ensure consecutive lows are lower than previous lows
  extrema = []
  ex_deque = deque(maxlen=K)
  for i, idx in enumerate(low_idx):
    if i == 0:
      ex_deque.append(idx)
      continue
    if lows[i] > lows[i-1]:
      ex_deque.clear()

    ex_deque.append(idx)
    if len(ex_deque) == K:
      extrema.append(ex_deque.copy())

  return extrema

def calcRSI(data, P=14):
  data['diff_close'] = data['Close'] - data['Close'].shift(1)
  data['gain'] = np.where(data['diff_close']>0, data['diff_close'], 0)
  data['loss'] = np.where(data['diff_close']<0, np.abs(data['diff_close']), 0)
  data[['init_avg_gain', 'init_avg_loss']] = data[
    ['gain', 'loss']].rolling(P).mean()
  avg_gain = np.zeros(len(data))
  avg_loss = np.zeros(len(data))
  for i, _row in enumerate(data.iterrows()):
    row = _row[1]
    if i < P - 1:
      last_row = row.copy()
      continue
    elif i == P-1:
      avg_gain[i] += row['init_avg_gain']
      avg_loss[i] += row['init_avg_loss']
    else:
      avg_gain[i] += ((P - 1) * avg_gain[i-1] + row['gain']) / P
      avg_loss[i] += ((P - 1) * avg_loss[i-1] + row['loss']) / P
          
    last_row = row.copy()
      
  data['avg_gain'] = avg_gain
  data['avg_loss'] = avg_loss
  data['RS'] = data['avg_gain'] / data['avg_loss']
  data['RSI'] = 100 - 100 / (1 + data['RS'])
  return data
def getHHIndex(data: np.array, order=5, K=2):
  extrema = getHigherHighs(data, order, K)
  idx = np.array([i[-1] + order for i in extrema])
  return idx[np.where(idx<len(data))]

def getLHIndex(data: np.array, order=5, K=2):
  extrema = getLowerHighs(data, order, K)
  idx = np.array([i[-1] + order for i in extrema])
  return idx[np.where(idx<len(data))]

def getLLIndex(data: np.array, order=5, K=2):
  extrema = getLowerLows(data, order, K)
  idx = np.array([i[-1] + order for i in extrema])
  return idx[np.where(idx<len(data))]

def getHLIndex(data: np.array, order=5, K=2):
  extrema = getHigherLows(data, order, K)
  idx = np.array([i[-1] + order for i in extrema])
  return idx[np.where(idx<len(data))]

def getPeaks(data, key='Close', order=5, K=2):
  vals = data[key].values
  hh_idx = getHHIndex(vals, order, K)
  lh_idx = getLHIndex(vals, order, K)
  ll_idx = getLLIndex(vals, order, K)
  hl_idx = getHLIndex(vals, order, K)

  data[f'{key}_highs'] = np.nan
  data[f'{key}_highs'][hh_idx] = 1
  data[f'{key}_highs'][lh_idx] = -1
  data[f'{key}_highs'] = data[f'{key}_highs'].ffill().fillna(0)
  data[f'{key}_lows'] = np.nan
  data[f'{key}_lows'][ll_idx] = 1
  data[f'{key}_lows'][hl_idx] = -1
  data[f'{key}_lows'] = data[f'{key}_highs'].ffill().fillna(0)
  return data
def RSIDivergenceStrategy(data, P=14, order=5, K=2):
  data = getPeaks(data, key='Close', order=order, K=K)
  data = calcRSI(data, P=P)
  data = getPeaks(data, key='RSI', order=order, K=K)

  position = np.zeros(data.shape[0])
  for i, (t, row) in enumerate(data.iterrows()):
    if np.isnan(row['RSI']):
        continue
    # If no position is on
    if position[i-1] == 0:
      # Buy if indicator to higher low and price to lower low
      if row['Close_lows'] == -1 and row['RSI_lows'] == 1:
        if row['RSI'] < 50:
          position[i] = 1
          entry_rsi = row['RSI'].copy()

      # Short if price to higher high and indicator to lower high
      elif row['Close_highs'] == 1 and row['RSI_highs'] == -1:
        if row['RSI'] > 50:
          position[i] = -1
          entry_rsi = row['RSI'].copy()

    # If current position is long
    elif position[i-1] == 1:
        if row['RSI'] < 50 and row['RSI'] < entry_rsi:
            position[i] = 1
  
    # If current position is short
    elif position[i-1] == -1:
        if row['RSI'] < 50 and row['RSI'] > entry_rsi:
            position[i] = -1

  data['position'] = position
  return calcReturns(data)

def calcReturns(df):
  # Helper function to avoid repeating too much code
  df['returns'] = df['Close'] / df['Close'].shift(1)
  df['log_returns'] = np.log(df['returns'])
  df['strat_returns'] = df['position'].shift(1) * df['returns']
  df['strat_log_returns'] = df['position'].shift(1) * df['log_returns']
  df['cum_returns'] = np.exp(df['log_returns'].cumsum()) - 1
  df['strat_cum_returns'] = np.exp(df['strat_log_returns'].cumsum()) - 1
  df['peak'] = df['cum_returns'].cummax()
  df['strat_peak'] = df['strat_cum_returns'].cummax()
  return df
def rsidiver():
    position_df = RSIDivergenceStrategy(data)
    positions = position_df['position'].tolist()
    rsi_divergent_signal=positions
    return rsi_divergent_signal

In [4]:
def findindi(Name,Real_name,Syear,Smonth,Sday,Eyear,Emonth,Eday):
    import datetime
    import pandas_datareader.data as web
    import matplotlib.pyplot as plt
    import numpy as np
    import talib as ta
    import pandas as pd
    import yfinance as yf
    from collections import deque
    global df_signal,data_macd,dfa1,renko_signal,obv,dfa1_all,Sdate,Edate,stoch_adx_signal,ADX,atr_signal,data_macd,MACD
	#preprocess data
    Sdate ="%s-%s-%s" % (Syear,Smonth,Sday)
    Edate ="%s-%s-%s" % (Eyear,Emonth,Eday)
    start=datetime.datetime(Syear,Smonth,Sday)
    end=datetime.datetime(Eyear,Emonth,Eday)
    Stockdata=yf.download(Name,start,end)
    dfa1=Stockdata
    Byear = Syear-1
    start=datetime.datetime(Byear,Smonth,Sday)
    end=datetime.datetime(Eyear,Emonth,Eday)
    Stockdata=yf.download(Name,start,end)
    dfa1_all=Stockdata
    #psar
    ticker = Name
    yfObj = yf.Ticker(ticker)
    dt = yfObj.history(start=Sdate, end=Edate)[['High', 'Low']].round(2)
    dt=dt.loc[Sdate:Edate]
    dt.reset_index(inplace=True)
    indic = PSAR()
    dt['PSAR'] = dt.apply(lambda x: indic.calcPSAR(x['High'], x['Low']), axis=1)
    dt['Trend'] = indic.trend_list
    indic._calcPSAR()
    psar_signal=[0]*len(dt['Trend'])
    for idx,i in enumerate(dt['Trend']):
        if (pd.isna(i) or idx==0):
        	psar_signal[idx]=0
       	else:
            if (dt['Trend'][idx-1]==0 and dt['Trend'][idx]==1):
                psar_signal[idx]=1
            elif(dt['Trend'][idx-1]==1 and dt['Trend'][idx]==0):
                psar_signal[idx]=-1
            else:
                psar_signal[idx]=0
    
    MA30=ta.MA(dfa1_all['Close'], timeperiod=30, matype=0)
    MA30=MA30.loc[Sdate:Edate]
    MA30=MA30.array
    dfa1['MA30']=MA30

    MA50=ta.MA(dfa1_all['Close'], timeperiod=50, matype=0)
    MA50=MA50.loc[Sdate:Edate]
    MA50=MA50.array
    dfa1['MA50']=MA50

    MA200=ta.MA(dfa1_all['Close'], timeperiod=200, matype=0)
    MA200=MA200.loc[Sdate:Edate]
    MA200=MA200.array
    dfa1['MA200']=MA200
    
    MA100=ta.MA(dfa1_all['Close'], timeperiod=100, matype=0)
    MA100=MA100.loc[Sdate:Edate]
    MA100=MA100.array
    dfa1['MA100']=MA100
      
    MA25=ta.MA(dfa1_all['Close'], timeperiod=25, matype=0)
    MA25=MA25.loc[Sdate:Edate]
    MA25=MA25.array
    dfa1['MA25']=MA25
    
    MA89=ta.MA(dfa1_all['Close'], timeperiod=89, matype=0)
    MA89=MA89.loc[Sdate:Edate]
    MA89=MA89.array
    dfa1['MA89']=MA89
    
    MA21=ta.MA(dfa1_all['Close'], timeperiod=21, matype=0)
    MA21=MA21.loc[Sdate:Edate]
    MA21=MA21.array
    dfa1['MA21']=MA21
    ma21_89_signal=[]
    ma25_89_signal=[]
    signal=0
    buy_price = []
    sell_price = []
    prices = dfa1['Close']
    for i in range(len(dfa1)):
        if dfa1['MA89'][i] > dfa1['MA25'][i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                ma25_89_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                ma25_89_signal.append(0)
        elif dfa1['MA89'][i] < dfa1['MA25'][i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                ma25_89_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                ma25_89_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            ma25_89_signal.append(0)
    signal=0
    for i in range(len(dfa1)):
        if dfa1['MA89'][i] > dfa1['MA21'][i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                ma21_89_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                ma21_89_signal.append(0)
        elif dfa1['MA89'][i] < dfa1['MA21'][i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                ma21_89_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                ma21_89_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            ma21_89_signal.append(0)
    #ma30_200_signal
    prices = dfa1['Close']
    buy_price = []
    sell_price = []
    ma30_200_signal = []
    signal = 0
    
    for i in range(len(dfa1)):
        if dfa1['MA30'][i] > dfa1['MA200'][i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                ma30_200_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                ma30_200_signal.append(0)
        elif dfa1['MA30'][i] < dfa1['MA200'][i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                ma30_200_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                ma30_200_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            ma30_200_signal.append(0)
            
    MA5=ta.MA(dfa1_all['Close'], timeperiod=5, matype=0)
    MA5=MA5.loc[Sdate:Edate]
    MA5=MA5.array
    dfa1['MA5']=MA5

    MA10=ta.MA(dfa1_all['Close'], timeperiod=10, matype=0)
    MA10=MA10.loc[Sdate:Edate]
    MA10=MA10.array
    dfa1['MA10']=MA10        
    #ma5_10_signal        
    buy_price = []
    sell_price = []
    signal = 0
    ma5_10_signal=[]
    
    for i in range(len(dfa1)):
        if dfa1['MA5'][i] > dfa1['MA10'][i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                ma5_10_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                ma5_10_signal.append(0)
        elif dfa1['MA5'][i] < dfa1['MA10'][i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                ma5_10_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                ma5_10_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            ma5_10_signal.append(0)
    #ma50_200_signal
    buy_price = []
    sell_price = []
    ma50_200_signal,ma50_100_signal = [],[]
    signal = 0

    for i in range(len(dfa1)):
        if dfa1['MA50'][i] > dfa1['MA200'][i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                ma50_200_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                ma50_200_signal.append(0)
        elif dfa1['MA50'][i] < dfa1['MA200'][i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                ma50_200_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                ma50_200_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            ma50_200_signal.append(0) 
            
    for i in range(len(dfa1)):
        if dfa1['MA50'][i] > dfa1['MA100'][i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                ma50_100_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                ma50_100_signal.append(0)
        elif dfa1['MA50'][i] < dfa1['MA100'][i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                ma50_100_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                ma50_100_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            ma50_100_signal.append(0)  
    #ma30_50_signal
    buy_price = []
    sell_price = []
    ma30_50_signal = []
    signal = 0        
    for i in range(len(dfa1)):
        if dfa1['MA30'][i] > dfa1['MA50'][i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                ma30_50_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                ma30_50_signal.append(0)
        elif dfa1['MA30'][i] < dfa1['MA50'][i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                ma30_50_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                ma30_50_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            ma30_50_signal.append(0)
            
    obv = ta.OBV(dfa1_all["Close"], dfa1_all["Volume"])
    obv_mean=obv.rolling(20).mean()
    obv_mean=obv_mean.loc[Sdate:Edate]
    obv=obv.loc[Sdate:Edate]
    
    obv_signal = []
    signal = 0
    for i in range(len(obv_mean)):
        if obv[i] > obv_mean[i]:
            if signal != 1:
                signal = 1
                obv_signal.append(signal)
            else:
                obv_signal.append(0)
        elif obv[i] < obv_mean[i]:
            if signal != -1:
                signal = -1
                obv_signal.append(signal)
            else:
                obv_signal.append(0)
        else:
            obv_signal.append(0)
    
    MA100=ta.MA(dfa1_all['Close'], timeperiod=100, matype=0)
    MA100=MA100.loc[Sdate:Edate]
    MA100=MA100.array
    dfa1['MA100']=MA100
    prices=dfa1['Close']
    buy_price = []
    sell_price = []
    signal = 0
    sma100_signal=[]
    for i in range(len(prices)):
        if prices[i]>MA100[i]:
            signal = 1
            sma100_signal.append(signal)
        else:
            signal=-1
            sma100_signal.append(signal)
    MA89=ta.MA(dfa1_all['Close'], timeperiod=89, matype=0)
    MA89=MA89.loc[Sdate:Edate]
    MA89=MA89.array
    dfa1['MA89']=MA89
    signal = 0
    sma89_signal=[]
    
    for i in range(len(prices)):
        if prices[i]>MA89[i]:
            signal = 1
            sma89_signal.append(signal)
        else:
            signal=-1
            sma89_signal.append(signal)
    signal = 0
    ma30_100_signal=[]        
    for i in range(len(dfa1)):
        if dfa1['MA30'][i] > dfa1['MA100'][i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                ma30_100_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                ma30_100_signal.append(0)
        elif dfa1['MA30'][i] < dfa1['MA100'][i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                ma30_100_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                ma30_100_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            ma30_100_signal.append(0)
    global macd
    macd, macdsignal, macdhist = ta.MACD(dfa1_all['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    MACD=macd.loc[Sdate:Edate]
    dfa1['MACD']=MACD
    dfa1['macdsignal']=macdsignal
    dfa1['macdhist']=macdhist

    data_macd=dfa1
    data=dfa1
    prices=dfa1['Open']
    buy_price = []
    sell_price = []
    macd_crossover_signal = []
    signal = 0

    for i in range(len(data)):
        if data_macd['MACD'][i] > data_macd['macdsignal'][i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                macd_crossover_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                macd_crossover_signal.append(0)
        elif data_macd['MACD'][i] < data_macd['macdsignal'][i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                macd_crossover_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                macd_crossover_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            macd_crossover_signal.append(0)
    macd_overzero_signal=[]
    for i in range(len(data)):
        if data_macd['MACD'][i] > 0:
            macd_overzero_signal.append(1)
        else:
            macd_overzero_signal.append(-1)
    macd_kongpop_signal=[]
    for i in range(len(data)):
        if data_macd['MACD'][i-1] <= -0.3 and data_macd['MACD'][i-1] >=-0.3:
            macd_kongpop_signal.append(1)
        elif data_macd['MACD'][i-1] >= 0.5 and data_macd['MACD'][i-1] <=0.5:
            macd_kongpop_signal.append(-1)
        else:
            macd_kongpop_signal.append(0)
            
    ADX=ta.ADX(dfa1_all['High'], dfa1_all['Low'], dfa1_all['Close'], timeperiod=14)
    ADX=ADX.loc[Sdate:Edate]
    ADX=ADX.array
    dfa1['ADX']=ADX
    
    plus_DMI = ta.PLUS_DM(dfa1_all['High'], dfa1_all['Low'], timeperiod=14)
    minus_DMI= ta.MINUS_DM(dfa1_all['High'], dfa1_all['Low'], timeperiod=14)
    plus_DMI=plus_DMI.loc[Sdate:Edate]
    minus_DMI=minus_DMI.loc[Sdate:Edate]
    plus_DMI=plus_DMI.array
    minus_DMI=minus_DMI.array
    dfa1['plus_DMI']=plus_DMI
    dfa1['minus_DMI']=minus_DMI
    
    prices, pdi, ndi, adx=dfa1['Close'], dfa1['plus_DMI'], dfa1['minus_DMI'], dfa1['ADX']
    buy_price = []
    sell_price = []
    adx_signal = []
    signal = 0
    
    for i in range(len(prices)):
        if adx[i-1] < 25 and adx[i] > 25 and pdi[i] > ndi[i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                adx_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                adx_signal.append(0)
        elif adx[i-1] < 25 and adx[i] > 25 and ndi[i] > pdi[i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                adx_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                adx_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            adx_signal.append(0)
    
    fastk, fastd = ta.STOCHRSI(dfa1_all['Close'], timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
    K=fastk.loc[Sdate:Edate]
    K=K.array
    dfa1['%K']=K
    D=fastd.loc[Sdate:Edate]
    D=D.array
    dfa1['%D']=D
    buy_price = []
    sell_price = []
    stoch_signal = []
    signal = 0
    a,b,c=dfa1['Close'], dfa1['%K'], dfa1['%D']
    for i in range(len(dfa1['Close'])):
        k=dfa1['%K']
        d=dfa1['%D']
        if k[i] < 20 and d[i] < 20 and k[i] < d[i]:
            if signal != 1:
                buy_price.append(dfa1['Close'][i])
                sell_price.append(np.nan)
                signal = 1
                stoch_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                stoch_signal.append(0)
        elif k[i] > 80 and d[i] > 80 and k[i] > d[i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(dfa1['Close'][i])
                signal = -1
                stoch_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                stoch_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            stoch_signal.append(0)
    stoch_signal_2=[] 
    for i in range(len(dfa1)):
        if k[i] > d[i] and k[i-1] < d[i-1] and k[i]>=80:
            stoch_signal_2.append(-1)
        elif k[i] < d[i] and k[i-1] > d[i-1] and k[i]<=20:
            stoch_signal_2.append(1)
        else:
            stoch_signal_2.append(0)
    import numpy as np
    import pandas_datareader as pdr
    import datetime as dt
    high_low = dfa1_all['High'] - dfa1_all['Low']
    high_close = np.abs(dfa1_all['High'] - dfa1_all['Close'].shift())
    low_close = np.abs(dfa1_all['Low'] - dfa1_all['Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)
    atr = true_range.rolling(14).sum()/14
    atr=atr.loc[Sdate:Edate]
    
    roc = ta.ROC(dfa1_all["Close"], timeperiod=10)
    roc=roc.loc[Sdate:Edate]
    data=roc
    roc_signal = []
    signal = 0
    for i in range(len(data)):
        if data[i] > 0:
            if signal != 1:
                signal = 1
                roc_signal.append(signal)
            else:
                roc_signal.append(0)
        elif data[i] < 0:
            if signal != -1:
                signal = -1
                roc_signal.append(signal)
            else:
                roc_signal.append(0)
        else:
            roc_signal.append(0)
   
    RSI = ta.RSI(dfa1_all['Close'], timeperiod=14)
    RSI=RSI.loc[Sdate:Edate]
    RSI=RSI.array
    dfa1['RSI']=RSI
    buy_price = []
    sell_price = []
    rsi_signal = []
    signal = 0
    prices, data=RSI, dfa1
    for i in range(len(prices)):
        if prices[i] <= 20:
            signal=1
        elif prices[i] >= 80:
            signal=-1
        else:
            signal=0
        rsi_signal.append(signal)
    CCI=ta.CCI(dfa1_all["High"],dfa1_all["Low"],dfa1_all["Close"],timeperiod=14)
    CCI=CCI.loc[Sdate:Edate]
    cci_signal = []
    data=CCI
    signal = 0
    countb,counts =0,0
    for i in range(len(data)):
        if data[i] <= -90:
            counts = 0
            countb = countb+1
            if countb == 6 and signal != 1 :
                signal = 1
                cci_signal.append(signal)
            else:
                cci_signal.append(0)
        elif data[i] >90:
            countb = 0
            counts = counts+1
            if counts == 6 and signal != -1:
                signal = -1
                cci_signal.append(signal)
            else:
                cci_signal.append(0)
        else:
            cci_signal.append(0)
            countb,counts =0,0
    atr = ta.ATR(dfa1_all["High"], dfa1_all["Low"], dfa1_all["Close"], timeperiod=14)
    atrmean= ta.MA(atr , timeperiod=14, matype=0)
    atr = atr.loc[Sdate:Edate]
    atrmean = atrmean.loc[Sdate:Edate]
    atr_signal = []
    data,data2=atr,atrmean
    signal = 0
    for i in range(len(data)):
        if data[i] > data2[i] :
            if signal != 1:
                signal = 1
                atr_signal.append(signal)
            else:
                atr_signal.append(0)
        elif data[i] < data2[i]:
            if signal != -1:
                signal = -1
                atr_signal.append(signal)
            else:
                atr_signal.append(0)
        else:
            atr_signal.append(0)
    wpr=ta.WILLR(dfa1_all["High"],dfa1_all["Low"],dfa1_all["Close"],timeperiod=14)
    wpr=wpr.loc[Sdate:Edate]
    wpr_signal = []
    data=wpr
    signal = 0
    countb,counts =0,0
    for i in range(len(data)):
        if data[i] <= -70:
            counts = 0
            if data[i]<= -80 :
                countb = countb+1
            if countb == 5 and signal != 1 :
                signal = 1
                wpr_signal.append(signal)
            else:
                wpr_signal.append(0)
        elif data[i] > -30:
            countb = 0
            if data[i]>= -20:
                counts = counts+1
            if counts == 5 and signal != -1:
                signal = -1
                wpr_signal.append(signal)
            else:
                wpr_signal.append(0)
        else:
            wpr_signal.append(0)
            countb,counts =0,0
    
    upperband, middleband, lowerband = ta.BBANDS(dfa1_all['Close'], timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    upperband, middleband, lowerband=upperband, middleband, lowerband.loc[Sdate:Edate]
    upperband, middleband, lowerband=upperband, middleband, lowerband.array
    dfa1['upperband']=upperband
    dfa1['middleband']=middleband
    dfa1['lowerband']=lowerband
    buy_price = []
    sell_price = []
    bb_signal = []
    signal = 0
    data=dfa1['Open']
    lower_bb=dfa1['lowerband']
    upper_bb=dfa1['upperband']
    a,b,c=dfa1['Close'], dfa1['lowerband'], dfa1['upperband']
    for i in range(len(data)):
        if data[i-1] > lower_bb[i-1] and data[i] < lower_bb[i]:
            if signal != 1:
                buy_price.append(data[i])
                sell_price.append(np.nan)
                signal = 1
                bb_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                bb_signal.append(0)
        elif data[i-1] < upper_bb[i-1] and data[i] > upper_bb[i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(data[i])
                signal = -1
                bb_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                bb_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            bb_signal.append(0)
    
    ohlc4=(dfa1_all['Close']+dfa1_all['Low']+dfa1_all['High']+dfa1_all['Open'])/4

    EMA2=ta.EMA(ohlc4, timeperiod=2)
    EMA26=ta.EMA(EMA2, timeperiod=26)
    EMA12=ta.EMA(EMA2, timeperiod=12)

    EMA26=EMA26.loc[Sdate:Edate]
    EMA26=EMA26.array
    dfa1['EMA26']=EMA26

    EMA12=EMA12.loc[Sdate:Edate]
    EMA12=EMA12.array
    dfa1['EMA12']=EMA12

    EMA2=EMA2.loc[Sdate:Edate]
    EMA2=EMA2.array
    dfa1['EMA2']=EMA2

    ohlc4=ohlc4.loc[Sdate:Edate]
    ohlc4=ohlc4.array
    dfa1['ohlc4']=ohlc4
    fastslow = []
    for i in range(len(dfa1)):
        if dfa1['EMA12'][i] > dfa1['EMA26'][i]:
            fastslow.append(1)
        elif dfa1['EMA26'][i] > dfa1['EMA12'][i]:
            fastslow.append(-1)
        else:
            fastslow.append(0)

    color = []
    for i in range(len(dfa1)):
        if fastslow[i] == 1 and dfa1['EMA2'][i] > dfa1['EMA12'][i]:
            color.append(1) 
        elif fastslow[i] == -1 and dfa1['EMA2'][i] > dfa1['EMA12'][i]:
            color.append(0.5)
        elif fastslow[i] == -1 and dfa1['EMA2'][i] < dfa1['EMA12'][i]:
            color.append(-1) 
        elif fastslow[i] == 1 and dfa1['EMA2'][i] < dfa1['EMA12'][i]:
            color.append(-0.5) 
        else:
            color.append(0)
    
    signal=0
    cdc_signal=[]
    for i in range (len(color)):
        if color[i] == 1:
            if signal != 1:
                signal = 1
                cdc_signal.append(signal)
            else:
                cdc_signal.append(0)
        elif color[i] == -1:
            if signal != -1:
                signal = -1
                cdc_signal.append(signal)
            else:
                cdc_signal.append(0)
        elif color[i] == 0.5:
            if signal != 0.5:
                signal = 0.5
                cdc_signal.append(signal)
            else:
                cdc_signal.append(0)
        elif color[i] == -0.5:
            if signal != -0.5:
                signal = -0.5
                cdc_signal.append(signal)
            else:
                cdc_signal.append(0)

        else:
            signal=0
            cdc_signal.append(signal)
    dfa1['cdc']=cdc_signal
    SAR=ta.SAR(dfa1_all['High'], dfa1_all['Low'], acceleration=0.02, maximum=0.2)
    SAR=SAR.loc[Sdate:Edate]
    SAR=SAR.array
    dfa1['SAR']=SAR
    signal= 0
    countb,counts =0,0
    sar_signal = []
    for i in range(len(dfa1)):
        if dfa1['Close'][i] > dfa1['SAR'][i]:
            counts = 0
            countb =countb +1
            if countb == 2:
                signal = 1
                sar_signal.append(signal)
            else:
                sar_signal.append(0)
        elif dfa1['Close'][i] < dfa1['SAR'][i]:
            countb = 0
            counts =counts +1
            if counts == 2:
                signal = -1
                sar_signal.append(signal)
            else:
                sar_signal.append(0)
        else:
            sar_signal.append(0)
    dfa1['sar']=sar_signal
    down,up=ta.AROON(dfa1_all['High'], dfa1_all['Low'], timeperiod=14)
    down=down.loc[Sdate:Edate]
    up=up.loc[Sdate:Edate]
    down=down.array
    up=up.array
    aroon_signal = []
    signal = 0
    for i in range(len(dfa1)):
        if up[i] >= 70 and down[i] <= 30:
            if signal != 1:
                signal = 1
                aroon_signal.append(signal)
            else:
                aroon_signal.append(0)
        elif up[i] <= 30 and down[i] >= 70:
            if signal != -1:
                signal = -1
                aroon_signal.append(signal)
            else:
                aroon_signal.append(0)
        else:
            aroon_signal.append(0)
    rsi2_signal=[]
    for i in range(len(dfa1)):
        if i!=0:
            if prices[i] <= 35 and macd_crossover_signal[i]==1 and prices[i]>prices[i-1]:
                rsi2_signal.append(1)
            elif prices[i] >= 65 and macd_crossover_signal[i]==-1 and prices[i]<=prices[i-1]:
                rsi2_signal.append(-1)
            else:
                rsi2_signal.append(0)
        else:
                rsi2_signal.append(0)

    renko_signal=[]
    renko_brick=[]
    renko_start=dfa1['Close'][0]
    for i in range(len(dfa1)):
        if i!=0:
            if dfa1['Close'][i]>=1.01*renko_start:
                renko_brick.append(1)
                renko_start=dfa1['Close'][i]
            elif dfa1['Close'][i]<=0.99*renko_start:
                renko_brick.append(-1)
                renko_start=dfa1['Close'][i]
            else:
                renko_brick.append(0)
            if renko_brick[i]==1 and renko_brick[i-1]==1:
                renko_signal.append(1)
            elif renko_brick[i]==-1 and renko_brick[i-1]==-1:
                renko_signal.append(-1)
            else:
                renko_signal.append(0)
        else:
            renko_brick.append(0)
            renko_signal.append(0)
    
    renko_macd_crossover_signal=[]
    for i in range(len(dfa1)):
        if i>=4:
            if renko_signal[i]==1 and data_macd['MACD'][i] > data_macd['macdsignal'][i] and data_macd['MACD'][i-1] > data_macd['macdsignal'][i-1] and data_macd['MACD'][i-2] > data_macd['macdsignal'][i-2] and data_macd['MACD'][i-3] > data_macd['macdsignal'][i-3] and data_macd['MACD'][i-4] > data_macd['macdsignal'][i-4]:
                renko_macd_crossover_signal.append(1)
            elif renko_signal[i]==-1 and data_macd['MACD'][i] < data_macd['macdsignal'][i] and data_macd['MACD'][i-1] < data_macd['macdsignal'][i-1] and data_macd['MACD'][i-2] < data_macd['macdsignal'][i-2]and data_macd['MACD'][i-3] < data_macd['macdsignal'][i-3]and data_macd['MACD'][i-4] < data_macd['macdsignal'][i-4]:
                renko_macd_crossover_signal.append(-1)
            else:
                renko_macd_crossover_signal.append(0)
        else:
            renko_macd_crossover_signal.append(0)
            
    obv_renko_signal=[]
    real=ta.LINEARREG_SLOPE(dfa1_all['Close'],timeperiod=14)
    real=real.loc[Sdate:Edate]
    for i in range(len(dfa1)):
        if real[i]<=0.3 and renko_signal[i]==1:
            obv_renko_signal.append(1)
        elif real[i]>0.3 and renko_signal[i]==-1:
            obv_renko_signal.append(-1)
        else:
            obv_renko_signal.append(0)
            
    obv_macd_crossover_signal=[]  
    for i in range(len(dfa1)):
        if real[i]<=0.3 and data_macd['MACD'][i] > data_macd['macdsignal'][i] and data_macd['MACD'][i-1] > data_macd['macdsignal'][i-1] and data_macd['MACD'][i-2] > data_macd['macdsignal'][i-2] and data_macd['MACD'][i-3] > data_macd['macdsignal'][i-3] and data_macd['MACD'][i-4] > data_macd['macdsignal'][i-4]:
            obv_macd_crossover_signal.append(1)
        elif real[i]>0.3 and data_macd['MACD'][i] < data_macd['macdsignal'][i] and data_macd['MACD'][i-1] < data_macd['macdsignal'][i-1] and data_macd['MACD'][i-2] < data_macd['macdsignal'][i-2]and data_macd['MACD'][i-3] < data_macd['macdsignal'][i-3]and data_macd['MACD'][i-4] < data_macd['macdsignal'][i-4]:
            obv_macd_crossover_signal.append(-1)
        else:
            obv_macd_crossover_signal.append(0)
    stoch_adx_signal=[]
    for i in range(len(dfa1)):
        if ADX[i]>35:
            if stoch_signal[i]==1:
                stoch_adx_signal.append(1)
            elif stoch_signal[i]==-1:
                stoch_adx_signal.append(-1)
            else:
                stoch_adx_signal.append(0)
        else:
            stoch_adx_signal.append(0)
    bb_rsi_signal=[]
    for i in range(len(data)):
        if (bb_signal[i]==1 and prices[i]>=50):
            bb_rsi_signal.append(1)
        elif (bb_signal[i]==-1 and prices[i]<=40):
            bb_rsi_signal.append(-1)
        else:
            bb_rsi_signal.append(0)
    psar_wpr_signal=[]
    for i in range(len(data)):
        if (psar_signal[i]==1 and wpr_signal[i]!=1):
            psar_wpr_signal.append(1)
       	elif (psar_signal[i]==-1 and wpr_signal[i]!=-1):
            psar_wpr_signal.append(-1)
        else:
            psar_wpr_signal.append(0)
    cdc_atr_signal=[]
    for i in range(len(data)):
        if (cdc_signal[i]==1 and atr_signal[i]==1):
            cdc_atr_signal.append(1)
       	elif (cdc_signal[i]==-1 and atr_signal[i]==1):
            cdc_atr_signal.append(-1)
        else:
            cdc_atr_signal.append(0)
        
    macd_sto_signal=[]
    for i in range(len(data)):
        if (stoch_signal[i]==1 and macd_crossover_signal[i]==1):
            macd_sto_signal.append(1)
       	elif (stoch_signal[i]==-1 and macd_crossover_signal[i]==-1):
            macd_sto_signal.append(-1)
        else:
            macd_sto_signal.append(0)
            
    sto_ma50_200_signal=[]
    for i in range(len(data)):
        if (stoch_signal[i]==1 and ma50_200_signal[i]==1):
            sto_ma50_200_signal.append(1)
       	elif (stoch_signal[i]==-1 and ma50_200_signal[i]==-1):
            sto_ma50_200_signal.append(-1)
        else:
            sto_ma50_200_signal.append(0)
            
    atr_adx_signal=[]
    for i in range(len(data)):
        if (atr_signal[i]==1 and adx_signal[i]==1):
            atr_adx_signal.append(1)
       	elif (atr_signal[i]==-1 and adx_signal[i]==-1):
            atr_adx_signal.append(-1)
        else:
            atr_adx_signal.append(0)
        
    data=rsiuse(dfa1)
    rsi_divergent_signal=rsidiver()
    
    signal={'adx_signal':adx_signal,'macd_crossover_signal':macd_crossover_signal,'ma30_200_signal':ma30_200_signal,'ma50_200_signal':ma50_200_signal,
            'ma30_50_signal':ma30_50_signal,'cdc_signal':cdc_signal,'sar_signal':sar_signal,
            'roc_signal':roc_signal,'obv_signal':obv_signal,'rsi_signal':rsi_signal,
            'bb_signal':bb_signal,'wpr_signal':wpr_signal,'stoch_signal':stoch_signal,
            'cci_signal':cci_signal,'atr_signal':atr_signal,'aroon_signal':aroon_signal,
            'sma100_signal':sma100_signal,
            'psar_signal':psar_signal,'stoch_signal_2':stoch_signal_2,'rsiandmacd':rsi2_signal,
            'renko_signal':renko_signal,'renko_macd_crossover_signal':renko_macd_crossover_signal,
            'obv_renko_signal':obv_renko_signal,'ma50_100_signal':ma50_100_signal,
            'obv_macd_crossover_signal':obv_macd_crossover_signal,'sma89_signal':sma89_signal,
            'ma30_100_signal':ma30_100_signal,'stoch_adx_signal':stoch_adx_signal,
            'bb_rsi_signal':bb_rsi_signal,'psar_wpr_signal':psar_wpr_signal,
            'cdc_atr_signal':cdc_atr_signal,'ma5_10_signal':ma5_10_signal,'macd_sto_signal':macd_sto_signal,
            'sto_ma50_200_signal':sto_ma50_200_signal,'atr_adx_signal':atr_adx_signal,'ma25_89_signal':ma25_89_signal,'ma21_89_signal':ma25_89_signal
            ,'macd_overzero_signal':macd_overzero_signal,'macd_kongpop_signal':macd_kongpop_signal,'rsi_divergent_signal':rsi_divergent_signal,
            'close_price':dfa1['Close']
           ,'adj_close_price':dfa1['Adj Close']}
    
    globals()['df_of_'+Real_name+'_signal'+'_train']=pd.DataFrame(signal)     

In [5]:
us_stock_list=['BA','FDX','AMP','COP','AAPL','TSLA','OXY','XOM','MCD','DIS']
us_stock_list+=['NFLX','PXD','AMZN','META','TROW','TSN','AMD','BABA']
us_stock_list+=['CRSP','EA','NKE','KO','INTC','F','CAT','SBUX','JNJ','QCOM','ADBE','ORCL']
ticker_list=[]
for i,value in enumerate(us_stock_list):
    ticker_list.append(us_stock_list[i])
for i in range(len(us_stock_list)):
    findindi(ticker_list[i],us_stock_list[i],2016,1,2,2023,3,9)
#globals()['df_of_'+us_stock_list[i]+'_signal_train'].to_csv(us_stock_list[i]+'_5_year_1519.csv')
ticker_list=[]
thai_stock_list=['CPALL','MINT','HANA', 'BTS','BDMS', 'FORTH','JASIF', 'SCC','PTT','KBANK']
thai_stock_list+=['AOT','CPF','CPN','LPN','IRPC','IT','ADVANC','BBL']
thai_stock_list+=['BANPU','COM7','GULF','JMART','TU','RCL','JAS','BGRIM','RBF','CHG','ORI','SINGER']
for i in range(len(thai_stock_list)):
     ticker_list.append(thai_stock_list[i]+'.BK')
for i,value in enumerate(thai_stock_list):
     findindi(ticker_list[i],thai_stock_list[i],2016,1,2,2023,3,9)
#     globals()['df_of_'+thai_stock_list[i]+'_signal_train'].to_csv(thai_stock_list[i]+'_5_year_1519.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


C:\Users\WINDOW~1\AppData\Local\Temp/ipykernel_15492/2071820884.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][hh_idx] = 1
C:\Users\WINDOW~1\AppData\Local\Temp/ipykernel_15492/2071820884.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][lh_idx] = -1
C:\Users\WINDOW~1\AppData\Local\Temp/ipykernel_15492/2071820884.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_lows'

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


C:\Users\WINDOW~1\AppData\Local\Temp/ipykernel_15492/2071820884.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][hh_idx] = 1
C:\Users\WINDOW~1\AppData\Local\Temp/ipykernel_15492/2071820884.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][lh_idx] = -1
C:\Users\WINDOW~1\AppData\Local\Temp/ipykernel_15492/2071820884.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_lows'

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

C:\Users\WINDOW~1\AppData\Local\Temp/ipykernel_15492/2071820884.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][hh_idx] = 1
C:\Users\WINDOW~1\AppData\Local\Temp/ipykernel_15492/2071820884.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][lh_idx] = -1
C:\Users\WINDOW~1\AppData\Local\Temp/ipykernel_15492/2071820884.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_lows'

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


C:\Users\WINDOW~1\AppData\Local\Temp/ipykernel_15492/2071820884.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][hh_idx] = 1
C:\Users\WINDOW~1\AppData\Local\Temp/ipykernel_15492/2071820884.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][lh_idx] = -1
C:\Users\WINDOW~1\AppData\Local\Temp/ipykernel_15492/2071820884.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_lows'

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


C:\Users\WINDOW~1\AppData\Local\Temp/ipykernel_15492/2071820884.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][hh_idx] = 1
C:\Users\WINDOW~1\AppData\Local\Temp/ipykernel_15492/2071820884.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][lh_idx] = -1
C:\Users\WINDOW~1\AppData\Local\Temp/ipykernel_15492/2071820884.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_lows'

In [10]:
all_stock_list=us_stock_list+thai_stock_list
for i,value in enumerate(all_stock_list):
    globals()[all_stock_list[i]+'_train'] = globals()['df_of_'+all_stock_list[i]+'_signal_train']
all_ticker_list=[]
for index,value in enumerate(all_stock_list):
    all_ticker_list.append(locals()[all_stock_list[index]+'_train'])
for i in range(60):
    dfo=all_ticker_list[i]
    globals()['sl1'+all_stock_list[i]]=dfo['adx_signal']
    globals()['sl2'+all_stock_list[i]]=dfo['macd_crossover_signal']
    globals()['sl3'+all_stock_list[i]]=dfo['ma30_200_signal']
    globals()['sl4'+all_stock_list[i]]=dfo['ma50_200_signal']
    globals()['sl5'+all_stock_list[i]]=dfo['ma30_50_signal']
    globals()['sl6'+all_stock_list[i]]=dfo['cdc_signal']
    globals()['sl7'+all_stock_list[i]]=dfo['sar_signal']
    globals()['sl8'+all_stock_list[i]]=dfo['roc_signal']
    globals()['sl9'+all_stock_list[i]]=dfo['obv_signal']
    globals()['sl10'+all_stock_list[i]]=dfo['rsi_signal']
    globals()['sl11'+all_stock_list[i]]=dfo['bb_signal']
    globals()['sl12'+all_stock_list[i]]=dfo['wpr_signal']
    globals()['sl13'+all_stock_list[i]]=dfo['stoch_signal']
    globals()['sl14'+all_stock_list[i]]=dfo['cci_signal']
    globals()['sl15'+all_stock_list[i]]=dfo['atr_signal']
    globals()['sl16'+all_stock_list[i]]=dfo['aroon_signal']
    globals()['sl17'+all_stock_list[i]]=dfo['sma100_signal']
    globals()['sl18'+all_stock_list[i]]=dfo['psar_signal']
    globals()['sl19'+all_stock_list[i]]=dfo['stoch_signal_2']
    globals()['sl20'+all_stock_list[i]]=dfo['rsiandmacd']
    globals()['sl21'+all_stock_list[i]]=dfo['renko_signal']
    globals()['sl22'+all_stock_list[i]]=dfo['renko_macd_crossover_signal']
    globals()['sl23'+all_stock_list[i]]=dfo['obv_renko_signal']
    globals()['sl24'+all_stock_list[i]]=dfo['ma50_100_signal']
    globals()['sl25'+all_stock_list[i]]=dfo['obv_macd_crossover_signal']
    globals()['sl26'+all_stock_list[i]]=dfo['sma89_signal']
    globals()['sl27'+all_stock_list[i]]=dfo['ma30_100_signal']
    globals()['sl28'+all_stock_list[i]]=dfo['stoch_adx_signal']
    globals()['sl29'+all_stock_list[i]]=dfo['bb_rsi_signal']
    globals()['sl30'+all_stock_list[i]]=dfo['psar_wpr_signal']
    globals()['sl31'+all_stock_list[i]]=dfo['cdc_atr_signal']
    globals()['sl32'+all_stock_list[i]]=dfo['ma5_10_signal']
    globals()['sl33'+all_stock_list[i]]=dfo['macd_sto_signal']
    globals()['sl34'+all_stock_list[i]]=dfo['sto_ma50_200_signal']
    globals()['sl35'+all_stock_list[i]]=dfo['atr_adx_signal']
    globals()['sl36'+all_stock_list[i]]=dfo['ma25_89_signal']
    globals()['sl37'+all_stock_list[i]]=dfo['ma21_89_signal']
    globals()['sl38'+all_stock_list[i]]=dfo['macd_overzero_signal']
    globals()['sl39'+all_stock_list[i]]=dfo['macd_kongpop_signal']
    globals()['sl40'+all_stock_list[i]]=dfo['rsi_divergent_signal']

In [11]:
for j in range(60): #เเต่ละหุ้น
    globals()['return_of_'+all_stock_list[j]+'_in_each_signal']=[]
def simulate_one_signal(all_ticker_list):
    global ret_l
    ret_l = [0] * 60
    for j in range(60): #เเต่ละหุ้น
        dfo=all_ticker_list[j]
        for i in range(1,41): #เเต่ละ signal
            sl=(globals()['sl'+str(i)+all_stock_list[j]])
            ret=0
            sumall=0
            amount=0
            mon=100000
            td=0.5
            for m in range(0,len(dfo)): #เเต่ละวัน
                r = sl[m]
                if r>td:
                    sumall=mon//dfo['adj_close_price'][m]
                    amount+=sumall
                    mon-=sumall*dfo['adj_close_price'][m]*1.002
                elif r<-td:
                    mon+=amount*dfo['adj_close_price'][m]*0.998
                    amount=0
                else:
                    pass
            ret=mon+amount*dfo['adj_close_price'][len(dfo)-1]
            globals()['return_of_'+all_stock_list[j]+'_in_each_signal'].append(((ret-100000)/1000))
            ret_l[i-1]+=ret

In [12]:
simulate_one_signal(all_ticker_list)
def Convert(lst,j):
    res_dct = {j[i]:lst[i] for i in range(1,41)}
    return res_dct

sorted_footballers_by_goals = sorted(Convert(ret_l,dfo.columns).items(), key=lambda x:(x[1]))
converted_dict = dict(sorted_footballers_by_goals)

print(converted_dict)

{'close_price': 0, 'macd_kongpop_signal': 6000000.0, 'atr_signal': 6562232.460411322, 'macd_sto_signal': 6651731.139663873, 'sto_ma50_200_signal': 7109070.7373552425, 'stoch_signal_2': 7341759.184944482, 'stoch_signal': 7595377.901186507, 'cci_signal': 7705486.853524061, 'wpr_signal': 8060558.600555265, 'rsi_signal': 8178899.4263981925, 'ma25_89_signal': 8385747.089073459, 'ma21_89_signal': 8385747.089073459, 'bb_signal': 8408796.393755702, 'rsi_divergent_signal': 8618438.073990643, 'rsiandmacd': 9213666.022689838, 'obv_signal': 10567978.784348905, 'ma5_10_signal': 11049458.101955341, 'renko_signal': 11227532.891993176, 'ma30_50_signal': 11230635.36441115, 'sar_signal': 11315847.227586336, 'atr_adx_signal': 11578632.738796504, 'stoch_adx_signal': 11746465.907470962, 'macd_crossover_signal': 12107229.29982879, 'psar_signal': 12157475.32734282, 'psar_wpr_signal': 12157475.32734282, 'cdc_atr_signal': 12333231.875065805, 'sma89_signal': 12410432.789595919, 'roc_signal': 12430752.424607184,

In [ ]:
for i in range(20):
	globals()['return_of_'+all_stock_list[i]+'_in_each_signal']

In [ ]:
len(return_of_BA_in_each_signal)

In [ ]:
#B&H return calculation
def buyandhold(all_ticker_list):
    global buyandhold_return_l
    buyandhold_return_l=[]
    for j in range(20): #เเต่ละหุ้น
        dfo=all_ticker_list[j]
        buyandhold_return_l.append(dfo['adj_close_price'][len(dfo)-1]-dfo['adj_close_price'][0])
buyandhold(all_ticker_list)

In [ ]:
signal_l=['atr_signal', 'stoch_signal_2', 'stoch_signal', 'cci_signal', 'rsi_signal', 'obv_signal', 'wpr_signal', 'bb_signal', 'sma100_signal', 'rsiandmacd', 'obv_renko_signal',
          'sma89_signal', 'ma30_100_signal', 'roc_signal', 'sar_signal', 'obv_macd_signal', 'stoch_adx_signal', 'psar_signal'
          , 'ma30_50_signal', 'cdc_signal', 'renko_signal', 'adx_signal', 'bb_rsi_signal' , 'macd_signal' , 'ma30_200_signal'
          , 'ma50_200_signal', 'ma50_100_signal', 'aroon_signal', 'renko_macd_signal']

In [ ]:
import pandas as pd
rows=[]
for j in range(20):
    rows.append(globals()['return_of_'+all_stock_list[j]+'_in_each_signal'])
#rows.append(buyandhold_return_l)
df = pd.DataFrame(rows, columns=signal_l)
df['B&H']=buyandhold_return_l
df['Stock']=all_stock_list

In [ ]:
df

In [ ]:
return_of_BA_in_each_signal

In [ ]:
start_date = '2018-01-01'
end_date = '2022-12-31'
ticker_name = "AAPL"
dfa1 = yf.download(ticker_name, start_date, end_date)

In [ ]:
real=ta.LINEARREG_SLOPE(dfa1_all['Close'],timeperiod=14)
real=real.loc[Sdate:Edate]

In [ ]:
Sdate

In [ ]:
real

In [ ]:
x

In [ ]:
obv = ta.OBV(dfa1_all["Close"], dfa1_all["Volume"])
obv_mean=obv.rolling(20).mean()
obv_mean=obv_mean.loc[Sdate:Edate]